## Setup

Note: not needed when reusable (database operations) code published on internal PyPI and pip installed into the env

In [1]:
import sys
from pathlib import Path

parent_path = str(Path.cwd())

if parent_path not in sys.path:
    sys.path.append(parent_path)

from db_operations.db import SQLite

In [2]:
db_file = 'fedimint-observer.db'

## 1. How much Bitcoin was pegged-in to all federations?

In [3]:
import pandas as pd

with SQLite(db_file) as db:
    df_transaction_inputs= pd.read_sql_query("SELECT * FROM transaction_inputs", db.conn)

In [4]:
import pandas as pd

def calculate_total_btc_pegged(transaction_inputs_or_outputs_df):
    """
    Calculate the total amount of Bitcoin pegged into or out each federation.

    Parameters:
        transaction_inputs_df (pd.DataFrame): DataFrame containing the transaction inputs or outputs.

    Returns:
        pd.DataFrame: A DataFrame with each federation and the corresponding total pegged-in or -out Bitcoin amount.
    """

    # Filter for peg-in transactions (assuming 'kind' == 'wallet' indicates peg-ins)
    filtered_df = transaction_inputs_or_outputs_df[transaction_inputs_or_outputs_df['kind'] == 'wallet']

    # Calculate the total amount of Bitcoin pegged-in per federation
    total_btc_per_federation = (
        filtered_df.groupby('federation_id')['amount_msat'].sum() / 100000000000
    ).reset_index()

    # Rename columns for clarity
    total_btc_per_federation.columns = ['federation_id', 'total_btc_pegged']

    return total_btc_per_federation

In [5]:
pegged_in = calculate_total_btc_pegged(df_transaction_inputs)

In [6]:
pegged_in

,federation_id,total_btc_pegged
0,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",8.423759


## 2. How much Bitcoin was pegged-out from all federations?

In [7]:
with SQLite(db_file) as db:
    df_transaction_outputs= pd.read_sql_query("SELECT * FROM transaction_outputs", db.conn)

In [8]:
pegged_out = calculate_total_btc_pegged(df_transaction_outputs)

In [9]:
pegged_out

,federation_id,total_btc_pegged
0,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",7.81929


## 3. What is the current on-chain balance of all federation?

In [10]:
def subtract_btc_pegged(df1, df2):
    # Merge the two DataFrames on `federation_id`
    merged_df = pd.merge(df1, df2, on='federation_id', suffixes=('_df1', '_df2'))

    # Subtract the `total_btc_pegged_in` columns
    merged_df['total_btc_difference'] = merged_df['total_btc_pegged_df1'] - merged_df['total_btc_pegged_df2']

    # Select relevant columns to return
    result_df = merged_df[['federation_id', 'total_btc_difference']]

    return result_df

result_df = subtract_btc_pegged(pegged_in, pegged_out)
result_df

,federation_id,total_btc_difference
0,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",0.604469


## And Freedom One?

In [11]:
# SQL query to find the federation with the name "Freedom One"
query = """
SELECT federation_id, config
FROM federations
WHERE config LIKE '%Freedom One%'
"""

with SQLite(db_file) as db:
    df = pd.read_sql_query(query, db.conn)

df

,federation_id,config


In [12]:
target_federation_id = df['federation_id'][0]

result_df = result_df[result_df['federation_id'] == target_federation_id]
result_df

KeyError: 0